# Markovify

This code is for training and testing the markov chain model for word recommendation

In [60]:
import markovify
import re
import spacy

!python -m spacy download en_core_web_sm

2023-04-24 21:30:15.344431: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Training the markov model based on our training dataset

In [75]:
with open("../data/output_test.txt") as f:
    text = f.read()
    
training_model = markovify.NewlineText(text, state_size=3)


In [99]:
print(training_model.make_short_sentence(180))

insane to watch a movie


Now we'll test spaCy's POS tagging with markovify to see if it improves text predictions.

In [81]:
nlp = spacy.load("en_core_web_sm")

class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

    def sentence_split(self, text):
        return re.split(r"\s*\n\s*", text)

In [83]:
training_model_2 = POSifiedText(text, state_size=3)

In [116]:
print(training_model_2.make_short_sentence(180))

what you gon na make fun of me
